# 自定义MRKL Agent

本笔记将介绍如何创建自定义的MRKL agent。

MRKL agent包括三个部分：

    - 工具：代理可用的工具。
    - LLMChain：生成以特定方式解析的文本的LLMChain。
    - 代理类本身：该类解析LLMChain的输出以确定要采取的动作。

本笔记将通过创建自定义LLMChain来演示如何创建自定义MRKL agent。

### 自定义LLMChain

创建自定义agent的一种方法是使用现有的Agent类，但使用自定义的LLMChain。这是创建自定义Agent的最简单方法。强烈建议您使用`ZeroShotAgent`，因为它目前是最通用的。

创建自定义LLMChain的大多数工作都取决于提示。因为我们使用现有的代理类来解析输出，所以提示说以那种格式生成文本非常重要。此外，我们目前需要一个`agent_scratchpad`输入变量，以记录以前的动作和观察结果的信息。这几乎总是提示的最后一部分。但是，除了这些说明之外，您可以根据需要自定义提示。

为了确保提示包含适当的说明，我们将使用该类上的一个帮助方法。 `ZeroShotAgent`的帮助方法接受以下参数：

- 工具：代理将访问的工具列表，用于格式化提示。
- prefix：放在工具列表之前的字符串。
- suffix：放在工具列表之后的字符串。
- input_variables：最终提示将期望的输入变量列表。

对于此练习，我们将使我们的代理能够访问Google搜索，并将其定制为以海盗身份回答。

In [1]:
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain import OpenAI, SerpAPIWrapper, LLMChain

In [2]:
search = SerpAPIWrapper()
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    )
]

In [3]:
prefix = """Answer the following questions as best you can, but speaking as a pirate might speak. You have access to the following tools:"""
suffix = """Begin! Remember to speak as a pirate when giving your final answer. Use lots of "Args"

Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools, 
    prefix=prefix, 
    suffix=suffix, 
    input_variables=["input", "agent_scratchpad"]
)

如果我们好奇，现在我们可以查看最终提示模板，以查看将其组合在一起时的样子。

In [4]:
print(prompt.template)

Answer the following questions as best you can, but speaking as a pirate might speak. You have access to the following tools:

Search: useful for when you need to answer questions about current events

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Search]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to speak as a pirate when giving your final answer. Use lots of "Args"

Question: {input}
{agent_scratchpad}


请注意，我们能够向代理提供自定义的提示模板，即不限于`create_prompt`函数生成的提示模板，假设它符合代理的要求。

例如，对于`ZeroShotAgent`，我们需要确保它满足以下要求：“操作：”开头的字符串，以及接下来的字符串以“操作输入：”开头，并且两者之间应该由换行符分隔。

In [5]:
llm_chain = LLMChain(llm=OpenAI(temperature=0), prompt=prompt)

In [6]:
tool_names = [tool.name for tool in tools]
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)

In [7]:
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)

In [8]:
agent_executor.run("How many people live in canada as of 2023?")



> Entering new AgentExecutor chain...
Thought: I need to find out the population of Canada
Action: Search
Action Input: Population of Canada 2023
Observation: The current population of Canada is 38,661,927 as of Sunday, April 16, 2023, based on Worldometer elaboration of the latest United Nations data.
Thought: I now know the final answer
Final Answer: Arrr, Canada be havin' 38,661,927 people livin' there as of 2023!

> Finished chain.


"Arrr, Canada be havin' 38,661,927 people livin' there as of 2023!"

### 多个输入
代理也可以使用需要多个输入的提示进行工作。

In [9]:
prefix = """Answer the following questions as best you can. You have access to the following tools:"""
suffix = """When answering, you MUST speak in the following language: {language}.

Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools, 
    prefix=prefix, 
    suffix=suffix, 
    input_variables=["input", "language", "agent_scratchpad"]
)

In [10]:
llm_chain = LLMChain(llm=OpenAI(temperature=0), prompt=prompt)

In [11]:
agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools)

In [12]:
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)

In [13]:
agent_executor.run(input="How many people live in canada as of 2023?", language="italian")



> Entering new AgentExecutor chain...
Thought: I should look for recent population estimates.
Action: Search
Action Input: Canada population 2023
Observation: 39,566,248
Thought: I should double check this number.
Action: Search
Action Input: Canada population estimates 2023
Observation: Canada's population was estimated at 39,566,248 on January 1, 2023, after a record population growth of 1,050,110 people from January 1, 2022, to January 1, 2023.
Thought: I now know the final answer.
Final Answer: La popolazione del Canada è stata stimata a 39.566.248 il 1° gennaio 2023, dopo un record di crescita demografica di 1.050.110 persone dal 1° gennaio 2022 al 1° gennaio 2023.

> Finished chain.


'La popolazione del Canada è stata stimata a 39.566.248 il 1° gennaio 2023, dopo un record di crescita demografica di 1.050.110 persone dal 1° gennaio 2022 al 1° gennaio 2023.'